To process multiple timeseries, an ID column is required.   
If timestamp column exists, its name is required.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
from io import StringIO
import importlib_resources

# import main from foresee.scripts
from foresee.scripts.main import prepare_fit_report


In [ ]:
# upload sample time-series dataframe with columns(id, date_stamp, y)

test_data_light_txt = importlib_resources.files('foresee.data').joinpath('test_data_light.csv').read_text()

ts_df = pd.read_csv(StringIO(test_data_light_txt))


ts_df['date_stamp'] = pd.to_datetime(ts_df['date_stamp'])
ts_df.head()


In [ ]:
# user defind parameters

# time series values column name: required if input dataframe has more than one column

endog_colname = 'y'

if len(ts_df.columns) > 1 and endog_colname is None:
    raise ValueError('time series column name is required!!!')

# time series frequency
freq = 5

# out of sample forecast length
fcst_length = 10

# available forecasting models
model_list = ['ewm_model', 'fft', 'holt_winters', 'prophet', 'sarimax']

# avilable output formats: 'best_model', 'all_best', 'all_models'
output_format = 'all_best'

# if comparing models (output_format is 'best_model' or 'all_best') then holdout length is required

if output_format == 'all_models':
    holdout_length = None
else:
    holdout_length = 20
    
# fit-forecast computations can be done in parallel for each time series. requires dask library!!!
# for sequential processing set fit_execution_method to 'non_parallel'

fit_execution_method = 'parallel'


# since we have two time series in this dataset, time series id column name and date-time column name are required.
gbkey = 'id'
ds_column = 'date_stamp'
tune = True

In [ ]:
'''
result:  dataframe containing fitted values and future forecasts
_:  list of dictionaries containing fitted values, forecasts, and errors (useful for debuging)
'''

result, _ = main.collect_result(
                                                    ts_df,
                                                    endog_colname,
                                                    ds_colname, 
                                                    gbkey,
                                                    freq, 
                                                    fcst_length, 
                                                    output_format, 
                                                    holdout_length, 
                                                    model_list,
                                                    fit_execution_method,
                                                    tune
                                            )

result.head()